In [1]:
import numpy as np
import random as random
import tkinter as tk
import matplotlib.pyplot as plt
from tkinter import Label

np.set_printoptions(threshold=np.inf)

master = tk.Tk()

#Cálculo del tablero
etiqueta1 = Label(master, text="Introduzca los parámetros deseados: ")
etiqueta1.grid(row=0)

tk.Label(master, 
         text="Tamaño del problema").grid(row=1)
tk.Label(master, 
         text="Estado objetivo").grid(row=2)
tk.Label(master, 
         text="Gamma [0, 1]").grid(row=3)
tk.Label(master, 
         text="Numero de episodios").grid(row=4)
tk.Label(master, 
         text="Parametro alpha [0, 1]").grid(row=5)
tk.Label(master, 
         text="Parametro epsilon [0, 1]").grid(row=6)
tk.Label(master, 
         text="Estado inicial (para el camino optimo)").grid(row=7)

e1 = tk.Entry(master)
e2 = tk.Entry(master)
e3 = tk.Entry(master)
e4 = tk.Entry(master)
e5 = tk.Entry(master)
e6 = tk.Entry(master)
e7 = tk.Entry(master)

e1.grid(row=1, column=1)
e2.grid(row=2, column=1)
e3.grid(row=3, column=1)
e4.grid(row=4, column=1)
e5.grid(row=5, column=1)
e6.grid(row=6, column=1)
e7.grid(row=7, column=1)

def inicializeRMatrix(numRows, numCols):
    return np.array([-1 for _ in range((numRows**2)*(numCols**2))]).reshape(numRows**2, numCols**2)

def createMatrix(m,tam):
    layer=0
    for i in range(tam):
        [m,layer]=addLayerToMatrix(m,i,layer)
    return m


def addLayerToMatrix(matrix, layer, count):
    for i in range(layer):
        
        matrix[layer][i] = int(count)
        count=count+1
        
    for i in range(layer+1):
        
        matrix[layer-i][layer] = int(count)
        count=count+1
    
    return [matrix, count]

#Este método obtiene automáticamente la Matriz R a partir del tablero (m) introducido por el usuario
def obtainRMatrix(m,size):
    num_col  = size
    num_fil  = size
    Rsize = size**2
    rMatrix = inicializeRMatrix(num_col,num_fil)
    res=[]
    for i in range(num_fil):
        for j in range(num_col):
            n0 = m[i][j]
            if 0<i<num_fil-1:
                rMatrix[m[i][j]][m[i-1][j]]=0
                rMatrix[m[i][j]][m[i+1][j]]=0
                
                if(0<j<num_col-1):
                    rMatrix[m[i][j]][m[i-1][j-1]]=0
                    rMatrix[m[i][j]][m[i][j-1]]=0
                    rMatrix[m[i][j]][m[i-1][j+1]]=0
                    rMatrix[m[i][j]][m[i+1][j-1]]=0
                    rMatrix[m[i][j]][m[i+1][j+1]]=0
                    rMatrix[m[i][j]][m[i][j+1]]=0
                    
                if(j==0):
                    rMatrix[m[i][j]][m[i-1][j+1]]=0
                    rMatrix[m[i][j]][m[i][j+1]]=0
                    rMatrix[m[i][j]][m[i+1][j+1]]=0
                    
                if(j==num_col-1):
                    rMatrix[m[i][j]][m[i-1][j-1]]=0
                    rMatrix[m[i][j]][m[i][j-1]]=0
                    rMatrix[m[i][j]][m[i+1][j-1]]=0  
                    
            elif(i==0):
                rMatrix[m[i][j]][m[i+1][j]]=0             
                
                if(j==0):
                    rMatrix[m[i][j]][m[i+1][j+1]]=0             
                    rMatrix[m[i][j]][m[i][j+1]]=0   
                    
                if(j==num_col-1):
                    rMatrix[m[i][j]][m[i+1][j-1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0 
                    
                if(0<j<num_col-1):
                    rMatrix[m[i][j]][m[i+1][j-1]]=0  
                    rMatrix[m[i][j]][m[i+1][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0  
                    
            elif i==num_fil-1:
                rMatrix[m[i][j]][m[i-1][j]]=0  
                if(j==0):
                    rMatrix[m[i][j]][m[i-1][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j+1]]=0  
                if(j==num_col-1):
                    rMatrix[m[i][j]][m[i-1][j-1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0  
                if(0<j<num_col-1):
                    rMatrix[m[i][j]][m[i-1][j+1]]=0  
                    rMatrix[m[i][j]][m[i-1][j-1]]=0  
                    rMatrix[m[i][j]][m[i][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0  
    return rMatrix

def addObjetives(R,objetives, size):
    for objetive in objetives:
        for i in range(size**2):
            R[i][objetive] = 100 if (R[i][objetive]==0) else -1       
        R[objetive][objetive] = 100
    return R

def entrenarQFase2 ( Q, R, Gamma, objetivo, num_episodios, size, alpha, epsilon ):
   #Convertir los valores alpha y epsilon a decimales
    alpha = float( alpha )
    epsilon = float( epsilon )
    rendimiento = []
    while num_episodios > 0:
        estado = random.randint ( 0, (size**2) - 1 )

        while ( estado != objetivo ):
            #Selección aleatoria del número que va a ser comparado con epsilon
            numero_aleatorio = random.uniform( 0, 1 ) 
            if numero_aleatorio < epsilon:
                #Número aleatorio < epsilon --> Acción aleatoria
                posiblesAcciones = np.asarray(acciones1(R,estado)).flatten()
                accion_elegida = np.random.choice (posiblesAcciones)
            else:
                #Número aleatorio >= epsilon --> Seleccionamos la acción con el máximo valor en la tabla Q
                qRow = Q [ estado ]
                rRow = R[ estado ]
                
                listaPosibles = takePossibleActionsWithMax(qRow, rRow)
                accion_elegida = np.random.choice (np.asarray(listaPosibles).flatten())
            siguiente_estado = accion_elegida
            Q [ estado, accion_elegida ] = R [ estado, accion_elegida ] + Gamma * maxQValue(Q,R,siguiente_estado)
            estado = siguiente_estado
            
        rendimiento.append((Q.sum()/np.max(Q))*100)
        num_episodios = num_episodios - 1
        epsilon = epsilon * alpha
    return Q, rendimiento

def maxQValue(Q, R, estado):
    #obtener los estados accesibles desde la matriz R
    rRow = R[estado]
    qRow = Q[estado]
    possibleStatesIndex = np.where(rRow != -1)
    #obtener los valores en la matriz Q
    qLista=qRow[possibleStatesIndex]
    #devolver el máximo
    maxValue=np.amax(qLista)
    return maxValue

def acciones ( matriz, estado ):
    fila_estado = matriz [ estado , : ]
    acciones = [ ]
    for n in range ( len ( fila_estado ) ):
        if (fila_estado [ n ] != -1):
            acciones.append(n)
    return (acciones)

def acciones1 ( matriz, estado ):
    rRow=matriz[estado]
    possibleStatesIndex = np.where(rRow != -1)
    acciones = possibleStatesIndex
    return (acciones)

#####Obtenemos los estados que cumplen la condicion qRow[i] == mavValue && rRow[i] != -1 ####
def takePossibleActionsWithMax(qRow, rRow):        
    maxValue = np.max( qRow )

    posiblesAccionesQ = np.where(qRow == maxValue)
    posiblesAccionesR = np.where(rRow != -1)

    listaPosibles = list(set(posiblesAccionesQ[0]) & set(posiblesAccionesR[0]))
    return listaPosibles

def findOptimalWay(initialState, Q, objetive):
    #paso1 
    actualState = initialState
    camino = [initialState]
    contador=0
    totalValue=0
    #paso2
    while(actualState!=objetive):
        qRow = Q[actualState]
        maxValue = np.max(qRow)
        maxValueIndex = np.where(qRow == maxValue)
        nextState=maxValueIndex[0][0]
        camino.append(nextState)
        #paso3
        totalValue += maxValue 
        actualState=nextState
        contador=contador+1
    return camino, contador, totalValue

def run():
    size = int(e1.get())
    objetive = int(e2.get())
    Gamma = float(e3.get())
    num_episodios = int(e4.get())
    alpha = float(e5.get())
    epsilon = float(e6.get())
    estado_inicial = int(e7.get())
    print("Datos escogidos para el problema: \n")
    print("Tamaño: " + str(size))
    print("Objetivo: " + str(objetive))
    print("Gamma: " + str(Gamma))
    print("Numero de iteraciones: " + str(num_episodios))
    print("Alpha: " + str(alpha))
    print("Epsilon: " + str(epsilon))
    print("Estado inicial: " + str(estado_inicial) + "\n")
    
    num_col  = size
    num_fil  = size
    rend = 0
    
    zeroMatrix= np.zeros((num_fil,num_col))
    tablero = createMatrix(zeroMatrix,size).astype(int)
    print("Tablero creado: \n")
    print(tablero)
    print("\n")
    R_base = obtainRMatrix(tablero,size)
    print("Matriz R: \n")
    print(R_base)
    print("\n")
    R_objetivos = addObjetives(R_base, [objetive], size)
    print("Matriz R con los objetivos añadidos: \n")
    print(R_objetivos)
    print("\n")
    Q_base = np.zeros((size**2,size**2)).astype(int)
    print("Matriz Q base (matriz de 0s): \n")
    print(Q_base)
    print("\n")
    [Q_entrenada,rend] = entrenarQFase2(Q_base, R_objetivos, Gamma, objetive, num_episodios, size, alpha, epsilon)
    print("Matriz Q entrenada tras "+str(num_episodios) + " episodios: \n")
    print(Q_entrenada)
    print("\n")
    print("Rendimiento de la matriz: " + str(rend[-1])+"\n")
    [camino,contador,totalValue]=findOptimalWay(estado_inicial, Q_entrenada, objetive)
    print("Camino recorrido: "+str(camino))
    print("Contador de saltos: "+str(contador))
    print("Valor total del camino: "+ str(totalValue))
    
tk.Button(master, 
      text='Enviar datos', command=run).grid(row=8, column=1, sticky=tk.W, pady=4)
tk.mainloop()

Datos escogidos para el problema: 

Tamaño: 3
Objetivo: 6
Gamma: 0.8
Numero de iteraciones: 10
Alpha: 0.5
Epsilon: 0.5
Estado inicial: 1

Tablero creado: 

[[0 3 8]
 [1 2 7]
 [4 5 6]]


Matriz R: 

[[-1  0  0  0 -1 -1 -1 -1 -1]
 [ 0 -1  0  0  0  0 -1 -1 -1]
 [ 0  0 -1  0  0  0  0  0  0]
 [ 0  0  0 -1 -1 -1 -1  0  0]
 [-1  0  0 -1 -1  0 -1 -1 -1]
 [-1  0  0 -1  0 -1  0  0 -1]
 [-1 -1  0 -1 -1  0 -1  0 -1]
 [-1 -1  0  0 -1  0  0 -1  0]
 [-1 -1  0  0 -1 -1 -1  0 -1]]


Matriz R con los objetivos añadidos: 

[[ -1   0   0   0  -1  -1  -1  -1  -1]
 [  0  -1   0   0   0   0  -1  -1  -1]
 [  0   0  -1   0   0   0 100   0   0]
 [  0   0   0  -1  -1  -1  -1   0   0]
 [ -1   0   0  -1  -1   0  -1  -1  -1]
 [ -1   0   0  -1   0  -1 100   0  -1]
 [ -1  -1   0  -1  -1   0 100   0  -1]
 [ -1  -1   0   0  -1   0 100  -1   0]
 [ -1  -1   0   0  -1  -1  -1   0  -1]]


Matriz Q base (matriz de 0s): 

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]